In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import Parsers as P

# Tutorial of the PwParser class

This tutorial describes the usage of the class PwParser of mppi used to extract information for the XML output file data-file-schema produced
by pw.

## Parse of a output file of Silicon 

__Follow the tutorial for the QeCalculator to produce the xml file used by the PwParser.__

The class is initialized by specifying the name of the xml file including its relative path 

In [3]:
results = P.PwParser('QeCalculator_test/ecut_40.save/data-file-schema.xml')

Parse file : QeCalculator_test/ecut_40.save/data-file-schema.xml


When the object is initialized several attributes are set, for instance

In [4]:
results.units

'Hartree atomic units'

In [5]:
print(results.natoms)
print(results.atomic_species)
print(results.atomic_positions)
print(results.nbands)
print(results.nkpoints)
print(results.spin_degen)

2
{'Si': ['2.808600000000000e1', 'Si.pbe-mt_fhi.UPF']}
[['Si', [-1.2875, 1.2875, 1.2875]], ['Si', [1.2875, -1.2875, -1.2875]]]
4
32
2


We can print the ks energies and weight for each kpoint

In [6]:
for k,e,w in zip(results.kpoints,results.evals,results.weights):
    print(k,e,w)

[0. 0. 0.] [-0.21237932  0.22480586  0.22480586  0.22480586] [0.00925926]
[-0.16666667  0.16666667 -0.16666667] [-0.19919102  0.13751175  0.20843984  0.20843984] [0.05555556]
[-0.33333333  0.33333333 -0.33333333] [-0.16220083  0.02987301  0.18835642  0.18835642] [0.05555556]
[ 0.5 -0.5  0.5] [-0.12757113 -0.02957086  0.18109825  0.18109825] [0.02777778]
[0.         0.33333333 0.        ] [-0.19470782  0.14932662  0.18151751  0.18151751] [0.05555556]
[-0.16666667  0.5        -0.16666667] [-0.16499801  0.06224724  0.15213876  0.16134166] [0.11111111]
[ 0.66666667 -0.33333333  0.66666667] [-0.12170161 -0.01655811  0.1235651   0.16017894] [0.11111111]
[ 0.5        -0.16666667  0.5       ] [-0.13567139  0.00519533  0.11383623  0.1775689 ] [0.11111111]
[3.33333333e-01 2.77555756e-17 3.33333333e-01] [-0.17775389  0.08840259  0.137224    0.20418049] [0.05555556]
[0.         0.66666667 0.        ] [-0.1429486   0.04085118  0.13689062  0.13689062] [0.05555556]
[ 0.83333333 -0.16666667  0.8333333

There are also several get methods, for instance

In [7]:
results.get_fermi()

6.117279040435398

The method get_evals  return an array with the ks energies for each kpoint. The energies are expressed in eV and the 
energy of VBM is used as reference. A gap, both direct or indirect, can be set. In this case the energies of the empty ks states is
shifted. This procedure __does not__ update the occupation levels of the empty states

In [8]:
results.get_evals(set_gap=1.2)[0]

There are no empty bands. `Set gap` has not been applied


array([-1.18964146e+01, -3.99369533e-08, -1.83300486e-09,  0.00000000e+00])

In [9]:
results.get_gap()

There are no empty states. Gap cannot be computed.


We test the PwParser using a nscf output file, in this case empty bands are present and the gap can be computed

In [14]:
result_nscf = P.PwParser('QeCalculator_test/bands_12.save/data-file-schema.xml')

Parse file : QeCalculator_test/bands_12.save/data-file-schema.xml


In [15]:
result_nscf.occupations[0]

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
result_nscf.get_gap()

Indirect gap system
Gap : 0.7363567574170764 eV
Direct gap : 2.5651333558109854 eV


{'gap': 0.7363567574170764,
 'direct_gap': 2.5651333558109854,
 'position_cbm': 12,
 'positon_vbm': 0}

In [19]:
result_nscf.get_evals(set_direct_gap=3.0)[0]

Apply a scissor of 0.43486664418901455 eV


array([-1.18964110e+01, -9.74240383e-07, -9.74234497e-07,  0.00000000e+00,
        3.00000000e+00,  3.00000056e+00,  3.00000056e+00,  3.58353139e+00,
        8.17161688e+00,  8.17161688e+00,  8.29564293e+00,  1.16527216e+01])

we see that in this case the energy of the 5-th states has been shift to 3 eV to implement the set_direct_gap requirement. 

It is also possible to add an explicit scissor to shift the empty bands

In [28]:
result_nscf.get_evals(set_scissor=1.)[0]

Apply a scissor of 1.0 eV


array([-1.18964110e+01, -9.74240383e-07, -9.74234497e-07,  0.00000000e+00,
        3.56513336e+00,  3.56513391e+00,  3.56513391e+00,  4.14866474e+00,
        8.73675023e+00,  8.73675023e+00,  8.86077629e+00,  1.22178550e+01])

We compute the transition energies from the full to empty bands. Here we show the transition
for the first k point

In [68]:
result_nscf.get_transitions(set_direct_gap=3.0)[0]

Apply a scissor of 0.43486664418901455 eV


array([14.89641099, 14.89641154, 14.89641154, 15.47994237, 20.06802786,
       20.06802786, 20.19205392, 23.54913261,  3.00000097,  3.00000153,
        3.00000153,  3.58353236,  8.17161785,  8.17161785,  8.29564391,
       11.6527226 ,  3.00000097,  3.00000153,  3.00000153,  3.58353236,
        8.17161785,  8.17161785,  8.29564391, 11.6527226 ,  3.        ,
        3.00000056,  3.00000056,  3.58353139,  8.17161688,  8.17161688,
        8.29564293, 11.65272162])

## Parse of a Graphene output file

__Run the Analysis_BandStructure notebook to produce the xml file used by the PwParser.__

We parse a graphene output file to test if systems without a gap are correctly parsed.

In [69]:
results_gra = P.PwParser('Pw_bands/graphene_nscf.save/data-file-schema.xml')

Parse file : Pw_bands/graphene_nscf.save/data-file-schema.xml


In [70]:
results_gra.occupations_kind

'smearing'

In [71]:
results_gra.occupations[0]

array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       4.22181265e-29, 6.89778699e-37, 1.76721673e-41, 1.28104982e-57])

In [72]:
results_gra.occupations[18] #the position of K

array([1. , 1. , 1. , 0.5, 0.5, 0. , 0. , 0. ])

In [73]:
results_gra.get_gap()

Direct gap system
Gap : 2.715685454290906e-10 eV


{'gap': 2.715685454290906e-10,
 'direct_gap': 2.715685454290906e-10,
 'position_cbm': 18,
 'positon_vbm': 18}